In [2]:
import os

os.getcwd()

'/media/hamza/Disque local/Hamza Bouajila/Portfolio/Building-a-Batch-ETL-Pipeline-RedPanda/notebooks'

In [3]:
os.chdir("../")
os.getcwd()

'/media/hamza/Disque local/Hamza Bouajila/Portfolio/Building-a-Batch-ETL-Pipeline-RedPanda'

In [7]:
import quixstreams as qx
from data_pipeline.src.core.config import settings
from data_pipeline.src.core.logging_config import setup_logging


def transform(msg):
    new_msg = []
    for i in range(len(msg["data"])):
        new_msg.append(msg["data"][i]["quote"]["USD"])
    return new_msg


class RedpandaInitializer:
    def __init__(self):
        self.logger = setup_logging("RedpandaInitializer")
        self.app = qx.Application(
            broker_address="localhost:19092",
            consumer_group="StreamingAppConsumerGroup",
            auto_offset_reset="earliest",
            loglevel="DEBUG",
        )

        self.input_topic = self.app.topic(
            settings.REDPANDA_TOPIC, value_deserializer="json"
        )
        self.output_topic = self.app.topic(
            "basic_cleaned_topic", value_serializer="json"
        )
        self.sdf = self.app.dataframe(self.input_topic)
        self.sdf = self.sdf.apply(transform)
        self.logger.info(f"Transformed data: {dir(self.sdf)}")
        self.sdf.to_topic(self.output_topic)

    async def async_run(self):
        await self.app.run()

In [8]:
redpanda = RedpandaInitializer()

In [9]:
await redpanda.async_run()

[2025-01-01 16:26:17,423] [INFO] [quixstreams] : Starting the Application with the config: broker_address="{'bootstrap.servers': 'localhost:19092'}" consumer_group="StreamingAppConsumerGroup" auto_offset_reset="earliest" commit_interval=5.0s commit_every=0 processing_guarantee="at-least-once"
[2025-01-01 16:26:17,424] [INFO] [quixstreams] : Topics required for this application: "StreamingApp", "basic_cleaned_topic"


KafkaException: KafkaError{code=_TRANSPORT,val=-195,str="Failed to get metadata: Local: Broker transport failure"}